In [24]:
pip install natasha ipymarkup

     |████████████████████████████████| 34.4MB 124kB/s 
     |████████████████████████████████| 51kB 3.7MB/s 
     |████████████████████████████████| 51kB 3.4MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26102 sha256=47444b470d353064b62e11b8d5897bbd2bdf9799447b782aca2a2fb41e16758b
  Stored in directory: /root/.cache/pip/wheels/f3/f2/66/e9c30d3e9499e65ea2fa0d07c002e64de63bd0adaa49c445bf
Successfully built intervaltree
  Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [25]:

from yargy import Parser, rule, or_, and_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display
from yargy.predicates import caseless, normalized, dictionary
from yargy.predicates import gte, lte


In [26]:
from google.colab import files 
uploaded = files.upload()

Saving avito_cars.txt to avito_cars.txt


In [27]:
pip install natasha

In [28]:
Cars = fact(
    'Cars',
    ['name', 'model', 'year', 'dv', 'prob']
)
YEAR = and_(gte(1990), lte(2021))

YEAR_WORDS = or_(
    rule(caseless('г'), '.'),
    rule(caseless('г'), '.', caseless('в'), '.'),
    rule(caseless('г'),  caseless('в')),
    rule(caseless('Г'), '.', caseless('В'), '.'),
    rule(normalized('год'),  normalized('выпуска')),
    rule(normalized('год'))
)
DV = or_(
    rule(and_(gte(1000), lte(2200))),
    rule(and_(gte(0.1), lte(0.9)))
    )

DV_WORDS = rule(normalized('двигатель'))

PROB_WORDS = or_(
    rule(caseless('Пробег'), '-'),
    rule(caseless('пробег'), '-'),
    rule(caseless('Пробег')),
    rule(caseless('ПРОБЕГ'), '-'),
    rule(normalized('пробег'))
)
PROB = rule(and_(gte(1000), lte(2200000)))

VAS = rule(
    morph_pipeline(['Ваз', 'ВАЗ', 'ваз', 'vas', 'vaz', 'VAS', 'VAZ']).interpretation(Cars.name),
    morph_pipeline(['21074', '2121', '2131', '2101', '2103', '2106', '21099', '1111']).interpretation(Cars.model.normalized()),
    or_(rule(YEAR), rule(YEAR, YEAR_WORDS.optional())).interpretation(Cars.year).optional(),
    rule(DV_WORDS.optional(), DV).interpretation(Cars.dv).optional(),
    rule(PROB_WORDS, PROB).interpretation(Cars.prob).optional()
    )

VOLVO = rule(
    morph_pipeline(['VOLVO', 'volvo', 'Volvo', 'Вольво', 'ВОЛЬВО', 'вольво']).interpretation(Cars.name),
    morph_pipeline(['ХС 70', 'xc70', 'xc 70', 'S40', 'хс 90', 'с 60', '850', 'S80', 's 80', '740', '760', '850', 'V90', 'xc40', 'XC40','xc60', 'XC60']).interpretation(Cars.model.normalized()),
    or_(rule(YEAR), rule(YEAR, YEAR_WORDS.optional())).interpretation(Cars.year).optional(),
    rule(DV_WORDS.optional(), DV).interpretation(Cars.dv).optional(),
    rule(PROB_WORDS, PROB).interpretation(Cars.prob).optional()
    )
CARS = or_(VAS, VOLVO).interpretation(Cars)

parser = Parser(CARS)

In [29]:
from razdel import sentenize

In [30]:
import pandas as pd

In [31]:
with open('avito_cars.txt') as f2:
  cars = f2.readlines()
df_c = pd.DataFrame({'text': cars})

In [ ]:
df_c

In [32]:
from tqdm import tqdm
matches = []

for sent in tqdm(df_c.text):
    for match in parser.findall(sent):
        matches.append(match.fact)

100%|██████████| 53650/53650 [03:02<00:00, 293.97it/s]


In [17]:
len(matches)

531

In [ ]:
for m in matches:
  print(m)